## **Multiple Linear Regression**

y=b0 + b1*x1 + .... + bn*xn

here dependent variable depends on multiple independentent variables.


### **Assumptions of Linear Regression:**
1.Linearity

2.Homoscedasticity

3.multivariate normality

4.Independence of errors

5.lack of multicollinearity

Before applying Linear regression, check this assumptions are true.

**Ex:Dataset>** R&D Spend,Administration,Marketing Spend,State,Profit
y=profit
x1=R&D Spend
x2=Administration
x3=Marketing Spend

since state is a categorical variable, we need to have dummy variable for this.ie:create a new columns for every category and keep 1/0 in it.ie:california,new york

D1=state  >if D1=1 then newyork else california.never include all dummy variable columns.this seems biased since california doesn't have its part in equation since D1=0, but inreality that's not the case because the way regression models work is that they will take by default.ie:the coefficient california is included in b0.

so, y=b0 + b1*x1 + b2*x2 + b3*x3 + b4*D1

<img


never include both dummy variables (california,new york).ie:b4*D1+b5*D2.because it means we are duplicating a variable.since always D2=1-D1 ,the phenomenon where one or several independent variables in a linear regression predict another is called multicollinearity as a result of this effect the model cannot distinguish between the effects of D-1 from the effects from of D2.so it won't work properly.This is called Dummy variable trap.

the real problem is that you cannot have these three b0,b4*D1,b5*D2 elements in your model at the same time the constant and both the dummy variables.

so,always omit one dummy variable and this applies irrespective of the number of dummy variables they are in that specific dummy set.

if there are 2 dummy sets made out of 2 different columns, then apply above for both the dummy sets.

### **P-Value:**
(for statistial significance)

in coin toss
H0(Null Hypothesis):Tis is a Fair coin
H1(Alternative Hypothesis):This is a not a Fair coin

assume H0 is true,based on the experiment see whether we can contradict it.

toss1>tails>0.5>50% probability

toss2>tails>0.25

toss3>tails>0.12

toss4>tails>0.06

toss5>tails>0.03>ie:if we assume H0 is true then probability of this happenning is only 3%

toss6>tails>0.01


in above experiment P-Value is droping.after alpha=0.05 we conclude that H0 id not true.ie:we are 5% sure that H0 is true and 95% sure H0 is not true.

alpha=confidence level (usually 95%).in medical we set to 99%.
<img3


### **Building a Model:**

If y depends on x1,x2,x3,x4,x5,x6,x7 , we should not take all the independent variables bacause 1.garbagein>garbageout 2.we need to explain why to use specific variable.so use only correct variable which help in predicting y.

**construct model:** 

Five methods of building model:

1.All-in:: using all variables beacause we know it is useful or may be intentional.it is also done for preparing for backward eleimination.

2.Backward Elimination:: 
  a)select significance level to stay in the model (Ex: SL=0.05)
  b)fit the full model with all possible predictors
  c)Consider predictor with highest P-Value.if P>SL, go to step d    otherwise FIN(finish ie:model is ready)
  d)Remove the predictor
  f)fir the model without this variable.goto step c

3.Forward Selection::
  a)select significance level to enter the model (Ex: SL=0.05)
  b)fit all the simple regression models y~xn.select one with lowest p-value.
  c)Keep this variable and fit all possible models with one extra predictor added to the one's you already have.
  d)consider the predictor with lowest P-value.if P<SL, go to step c   otherwise FIN(finish ie:Keep the previous model)


4.Bidirectional Elimination::
  a)select significance level to enter and to stay in the model (Ex: SLENTER=0.05,SLSTAY=0.05)
  b)perform the next step of forward selection (new variables must have P<SLENTER to enter)
  c)perform all the steps of backward elimination (old variables must have P<SLSTAY to stay).goto step b
  d)no variables can enter  and no old variables can exit.so FIN(model is ready)

5.All Possible models(Score Comparison)::
  a)select a criterion of goodness of fit(Ex:Akaike criterion)
  b)construct all possible regression models. (2^N) - 1 total combinations.
  Ex:if we have 10 columns in dataset,then its (2^10)-1=1023 models
  c)select the one with best criterion
  d)FINISH(model is ready)

2,3,4 refers to Stepwise Regression

In [ ]:
#given dataset from of R&D Spend,Administration,Marketing Spend,State,Profit, predict which is viable startup to invest fund?
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Import Dataset

In [31]:
dataset=pd.read_csv('https://raw.githubusercontent.com/ProgramSKAN/Sample-Data-For-Machine-Learning/master/50_Startups.csv')
X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

In [32]:
print(X)

[[165349.2 136897.8 471784.1 'New York']
 [162597.7 151377.59 443898.53 'California']
 [153441.51 101145.55 407934.54 'Florida']
 [144372.41 118671.85 383199.62 'New York']
 [142107.34 91391.77 366168.42 'Florida']
 [131876.9 99814.71 362861.36 'New York']
 [134615.46 147198.87 127716.82 'California']
 [130298.13 145530.06 323876.68 'Florida']
 [120542.52 148718.95 311613.29 'New York']
 [123334.88 108679.17 304981.62 'California']
 [101913.08 110594.11 229160.95 'Florida']
 [100671.96 91790.61 249744.55 'California']
 [93863.75 127320.38 249839.44 'Florida']
 [91992.39 135495.07 252664.93 'California']
 [119943.24 156547.42 256512.92 'Florida']
 [114523.61 122616.84 261776.23 'New York']
 [78013.11 121597.55 264346.06 'California']
 [94657.16 145077.58 282574.31 'New York']
 [91749.16 114175.79 294919.57 'Florida']
 [86419.7 153514.11 0.0 'New York']
 [76253.86 113867.3 298664.47 'California']
 [78389.47 153773.43 299737.29 'New York']
 [73994.56 122782.75 303319.26 'Florida']
 [67532

### Encoding categorical data

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[3])],remainder='passthrough') # apply onehotencoder to state(categorical) column.the encoded data will be at the beginning
X=np.array(ct.fit_transform(X))

In [34]:
print(X)

[[0.0 0.0 1.0 165349.2 136897.8 471784.1]
 [1.0 0.0 0.0 162597.7 151377.59 443898.53]
 [0.0 1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 0.0 1.0 144372.41 118671.85 383199.62]
 [0.0 1.0 0.0 142107.34 91391.77 366168.42]
 [0.0 0.0 1.0 131876.9 99814.71 362861.36]
 [1.0 0.0 0.0 134615.46 147198.87 127716.82]
 [0.0 1.0 0.0 130298.13 145530.06 323876.68]
 [0.0 0.0 1.0 120542.52 148718.95 311613.29]
 [1.0 0.0 0.0 123334.88 108679.17 304981.62]
 [0.0 1.0 0.0 101913.08 110594.11 229160.95]
 [1.0 0.0 0.0 100671.96 91790.61 249744.55]
 [0.0 1.0 0.0 93863.75 127320.38 249839.44]
 [1.0 0.0 0.0 91992.39 135495.07 252664.93]
 [0.0 1.0 0.0 119943.24 156547.42 256512.92]
 [0.0 0.0 1.0 114523.61 122616.84 261776.23]
 [1.0 0.0 0.0 78013.11 121597.55 264346.06]
 [0.0 0.0 1.0 94657.16 145077.58 282574.31]
 [0.0 1.0 0.0 91749.16 114175.79 294919.57]
 [0.0 0.0 1.0 86419.7 153514.11 0.0]
 [1.0 0.0 0.0 76253.86 113867.3 298664.47]
 [0.0 0.0 1.0 78389.47 153773.43 299737.29]
 [0.0 1.0 0.0 73994.56 122782.75 3

### Splitting Dataset into Train and Test Set

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

**Feature Scaling** is not required in multiple linear regression because in the equation of the multiple linear regression we have coefficients that is multiplied to each independent viable of each feature and therefore it doesn't matter that some features have higher values than others because the coefficients will compensate to put everything on the same scale.

We dont need to check all the assumptions of multiple linear reggression in the interest of time.Instead we check multiple models whether it is giving better result or not

### Training Multiple Linear Regression on the Training Set

In [38]:
from sklearn.linear_model import LinearRegression#dummy variable trap will indeed avoided by this class itself
#backward elimination technique is also done by the class.this class will  automatically identify the best features of the features that have the highest p values or that are the most statistically significant to figure out how to predict.

regressor=LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Predicting Test Set Results

In [42]:
#not plotting since 4 features,so we need 5D
y_pred=regressor.predict(X_test)
np.set_printoptions(precision=2) #this will display any numerical value with only 2 decimals after ,
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),axis=1)) #axis=0 vertical concatination,axis=1 horizantal concatination
#[predicted profit,actual profit]

[[103015.2  103282.38]
 [132582.28 144259.4 ]
 [132447.74 146121.95]
 [ 71976.1   77798.83]
 [178537.48 191050.39]
 [116161.24 105008.31]
 [ 67851.69  81229.06]
 [ 98791.73  97483.56]
 [113969.44 110352.25]
 [167921.07 166187.94]]


Making a single prediction (for example the profit of a startup with R&D Spend = 160000, Administration Spend = 130000, Marketing Spend = 300000 and State = 'California')

In [45]:
print(regressor.predict([[1, 0, 0, 160000, 130000, 300000]]))

[181566.92]


Therefore, our model predicts that the profit of a Californian startup which spent 160000 in R&D, 130000 in Administration and 300000 in Marketing is $ 181566,92.

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array. Simply put:

$1, 0, 0, 160000, 130000, 300000 \rightarrow \textrm{scalars}$

$[1, 0, 0, 160000, 130000, 300000] \rightarrow \textrm{1D array}$

$[[1, 0, 0, 160000, 130000, 300000]] \rightarrow \textrm{2D array}$

**Important note 2:** Notice also that the "California" state was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the second row of the matrix of features X, "California" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, not the last three ones, because the dummy variables are always created in the first columns.

### Getting the final linear regression equation with the values of the coefficients

In [46]:
print(regressor.coef_)
print(regressor.intercept_)

[ 8.66e+01 -8.73e+02  7.86e+02  7.73e-01  3.29e-02  3.66e-02]
42467.52924853204


Therefore, the equation of our multiple linear regression model is:

$$\textrm{Profit} = 86.6 \times \textrm{Dummy State 1} - 873 \times \textrm{Dummy State 2} + 786 \times \textrm{Dummy State 3} - 0.773 \times \textrm{R&D Spend} + 0.0329 \times \textrm{Administration} + 0.0366 \times \textrm{Marketing Spend} + 42467.53$$

**Important Note:** To get these coefficients we called the "coef_" and "intercept_" attributes from our regressor object. Attributes in Python are different than methods and usually return a simple value or an array of values.

predicted profit,actual profit is closer.to apply backward corelation manually to build model,check below code.

In [44]:
#Manually applying Backwar corelation
# # Multiple Linear Regression
 
# # Importing the libraries
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
 
# # Importing the dataset
# dataset = pd.read_csv('50_Startups.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
# print(X)
 
# # Encoding categorical data
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))
# print(X)
 
# # Avoiding the Dummy Variable Trap
# X = X[:, 1:]
 
# # Splitting the dataset into the Training set and Test set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
 
# # Training the Multiple Linear Regression model on the Training set
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)
 
# # Predicting the Test set results
# y_pred = regressor.predict(X_test)
# np.set_printoptions(precision=2)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
 
# # Building the optimal model using Backward Elimination
# import statsmodels.api as sm
# X = np.append(arr = np.ones((50, 1)).astype(int), values = X, axis = 1)
# X_opt = X[:, [0, 1, 2, 3, 4, 5]]
# X_opt = X_opt.astype(np.float64)
# regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
# regressor_OLS.summary()X_opt = X[:, [0, 1, 3, 4, 5]]
# X_opt = X_opt.astype(np.float64)
# regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
# regressor_OLS.summary()X_opt = X[:, [0, 3, 4, 5]]
# X_opt = X_opt.astype(np.float64)
# regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
# regressor_OLS.summary()X_opt = X[:, [0, 3, 5]]
# X_opt = X_opt.astype(np.float64)
# regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
# regressor_OLS.summary()X_opt = X[:, [0, 3]]
# X_opt = X_opt.astype(np.float64)regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
# regressor_OLS.summary()